In [1]:
import pandas as pd
import datetime as dt

import database as db
import ib_trades as ibt

/tmp/ipykernel_495581/4259107160.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Global variables for database and table names
DB_FILE = '/home/allay/servershare/trading_data.db'
TABLE_NAME = 'trades'
CREATE_TABLE = True
QUERY = 957448361226423544216934
QUERY_ID = 904255

In [3]:
# get today's date and store in a variable
today = dt.date.today()
trades_data_file = '/home/allay/servershare/'+today.strftime('%Y-%m-%d')+'_ibkr.xml'

In [4]:
# Download the trades data from IBKR
ibt.download_trades(trades_data_file, QUERY, QUERY_ID)

trades = ibt.extract_trades_from_file(trades_data_file)
df_trades = ibt.parse_trades_to_df(trades)

In [5]:
print(f'Imported {df_trades.shape[0]} transactions with {df_trades.shape[1]} attributes')

Imported 119 transactions with 32 attributes


In [6]:
df_trades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   accountId             119 non-null    object        
 1   currency              119 non-null    object        
 2   dateTime              119 non-null    datetime64[ns]
 3   assetCategory         119 non-null    object        
 4   symbol                119 non-null    object        
 5   description           119 non-null    object        
 6   conid                 119 non-null    object        
 7   listingExchange       116 non-null    object        
 8   multiplier            119 non-null    object        
 9   strike                59 non-null     object        
 10  expiry                59 non-null     object        
 11  putCall               119 non-null    object        
 12  tradeID               119 non-null    object        
 13  transactionID       

In [7]:
df_trades.iloc[0]

accountId                          U6446960
currency                                USD
dateTime                2024-02-29 16:01:43
assetCategory                         STOCK
symbol                                 AMZN
description                  AMAZON.COM INC
conid                               3691937
listingExchange                      NASDAQ
multiplier                                1
strike                                 None
expiry                                 None
putCall                                None
tradeID                           383380988
transactionID                    2004415726
reportDate                       2024-02-29
tradeDate                        2024-02-29
exchange                             ISLAND
quantity                                  2
tradePrice                           176.67
netCash                             -354.34
buySell                                 BUY
ibOrderID                         238795384
openCloseIndicator              

In [8]:
# Create a database connection
conn = db.create_connection(DB_FILE)

In [16]:
def insert_trade_row(conn, row):
    """
    Insert a single row into the trades table, ensuring all data types match the table schema.
    """
    sql_insert_trade = ''' INSERT INTO trades(accountId, currency, dateTime, assetCategory, symbol, description, conid,
                            listingExchange, multiplier, strike, expiry, putCall, tradeID, transactionID, reportDate,
                            tradeDate, exchange, quantity, tradePrice, netCash, buySell, ibOrderID, openCloseIndicator,
                            orderType, cost, fifoPnlRealized, mtmPnl, underlyingSymbol, underlyingConid, isAPIOrder,
                            ibCommission, ibCommissionCurrency)
                          VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?) '''

    # Convert dateTime, reportDate, and tradeDate to strings, and ensure numeric fields are properly formatted
    formatted_row = row.copy()
    formatted_row['dateTime'] = row['dateTime'].strftime('%Y-%m-%d %H:%M:%S') if row['dateTime'] else None
    formatted_row['reportDate'] = row['reportDate'].strftime('%Y-%m-%d') if row['reportDate'] else None
    formatted_row['tradeDate'] = row['tradeDate'].strftime('%Y-%m-%d') if row['tradeDate'] else None

    # Numeric fields conversion
    numeric_fields = ['multiplier', 'strike', 'quantity', 'tradePrice', 'netCash', 'cost', 'fifoPnlRealized', 'mtmPnl', 'ibCommission']
    for field in numeric_fields:
        if formatted_row[field] is not None:
            try:
                formatted_row[field] = float(formatted_row[field])
            except ValueError:
                formatted_row[field] = None  # or some default value if appropriate

    trade_data = (
        formatted_row['accountId'], formatted_row['currency'], formatted_row['dateTime'], formatted_row['assetCategory'], formatted_row['symbol'], formatted_row['description'],
        formatted_row['conid'], formatted_row['listingExchange'], formatted_row['multiplier'], formatted_row['strike'], formatted_row['expiry'], formatted_row['putCall'],
        formatted_row['tradeID'], formatted_row['transactionID'], formatted_row['reportDate'], formatted_row['tradeDate'], formatted_row['exchange'], formatted_row['quantity'],
        formatted_row['tradePrice'], formatted_row['netCash'], formatted_row['buySell'], formatted_row['ibOrderID'], formatted_row['openCloseIndicator'],
        formatted_row['orderType'], formatted_row['cost'], formatted_row['fifoPnlRealized'], formatted_row['mtmPnl'], formatted_row['underlyingSymbol'],
        formatted_row['underlyingConid'], str(formatted_row['isAPIOrder']), formatted_row['ibCommission'], formatted_row['ibCommissionCurrency']
    )
    
    try:
        c = conn.cursor()
        c.execute(sql_insert_trade, trade_data)
        conn.commit()
        print("Trade row inserted successfully.")
    except sqlite3.Error as e:
        print(e)


In [12]:
import sqlite3

In [17]:
insert_trade_row(conn, df_trades.iloc[0])

Trade row inserted successfully.


In [18]:
def fetch_last_trade_row(conn):
    """
    Fetch and print the most recently added row in the trades table.
    """
    sql_query = ''' SELECT * FROM trades ORDER BY id DESC LIMIT 1; '''
    
    try:
        c = conn.cursor()
        c.execute(sql_query)
        row = c.fetchone()
        if row:
            # Assuming you want to print the row. Adjust column names as necessary.
            print("Last added trade row:")
            columns = ['id', 'accountId', 'currency', 'dateTime', 'assetCategory', 'symbol', 'description', 'conid',
                       'listingExchange', 'multiplier', 'strike', 'expiry', 'putCall', 'tradeID', 'transactionID', 
                       'reportDate', 'tradeDate', 'exchange', 'quantity', 'tradePrice', 'netCash', 'buySell', 
                       'ibOrderID', 'openCloseIndicator', 'orderType', 'cost', 'fifoPnlRealized', 'mtmPnl', 
                       'underlyingSymbol', 'underlyingConid', 'isAPIOrder', 'ibCommission', 'ibCommissionCurrency']
            for i, column in enumerate(columns):
                print(f"{column}: {row[i]}")
        else:
            print("No rows found in the trades table.")
    except sqlite3.Error as e:
        print(e)

# Example usage
# Assuming 'conn' is your database connection
# fetch_last_trade_row(conn)


In [19]:
fetch_last_trade_row(conn)

Last added trade row:
id: 1
accountId: U6446960
currency: USD
dateTime: 2024-02-29 16:01:43
assetCategory: STOCK
symbol: AMZN
description: AMAZON.COM INC
conid: 3691937
listingExchange: NASDAQ
multiplier: 1.0
strike: None
expiry: None
putCall: None
tradeID: 383380988
transactionID: 2004415726
reportDate: 2024-02-29
tradeDate: 2024-02-29
exchange: ISLAND
quantity: 2.0
tradePrice: 176.67
netCash: -354.34
buySell: BUY
ibOrderID: 238795384
openCloseIndicator: OPEN
orderType: LIMIT
cost: 354.34
fifoPnlRealized: 0.0
mtmPnl: 0.18
underlyingSymbol: None
underlyingConid: None
isAPIOrder: False
ibCommission: -1.0
ibCommissionCurrency: USD


In [20]:
def fetch_all_trade_rows(conn):
    """
    Fetch and print all rows in the trades table.
    """
    sql_query = ''' SELECT * FROM trades; '''
    
    try:
        c = conn.cursor()
        c.execute(sql_query)
        rows = c.fetchall()
        if rows:
            print("All trade rows:")
            columns = ['id', 'accountId', 'currency', 'dateTime', 'assetCategory', 'symbol', 'description', 'conid',
                       'listingExchange', 'multiplier', 'strike', 'expiry', 'putCall', 'tradeID', 'transactionID', 
                       'reportDate', 'tradeDate', 'exchange', 'quantity', 'tradePrice', 'netCash', 'buySell', 
                       'ibOrderID', 'openCloseIndicator', 'orderType', 'cost', 'fifoPnlRealized', 'mtmPnl', 
                       'underlyingSymbol', 'underlyingConid', 'isAPIOrder', 'ibCommission', 'ibCommissionCurrency']
            for row in rows:
                row_data = [f"{column}: {value}" for column, value in zip(columns, row)]
                print(", ".join(row_data))
        else:
            print("No rows found in the trades table.")
    except sqlite3.Error as e:
        print(e)

In [21]:
fetch_all_trade_rows(conn)

All trade rows:
id: 1, accountId: U6446960, currency: USD, dateTime: 2024-02-29 16:01:43, assetCategory: STOCK, symbol: AMZN, description: AMAZON.COM INC, conid: 3691937, listingExchange: NASDAQ, multiplier: 1.0, strike: None, expiry: None, putCall: None, tradeID: 383380988, transactionID: 2004415726, reportDate: 2024-02-29, tradeDate: 2024-02-29, exchange: ISLAND, quantity: 2.0, tradePrice: 176.67, netCash: -354.34, buySell: BUY, ibOrderID: 238795384, openCloseIndicator: OPEN, orderType: LIMIT, cost: 354.34, fifoPnlRealized: 0.0, mtmPnl: 0.18, underlyingSymbol: None, underlyingConid: None, isAPIOrder: False, ibCommission: -1.0, ibCommissionCurrency: USD
